<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/simpsonsopencv%20dream_V3_newpix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r /content/*

In [ ]:

######################################################
from google.colab import drive
drive.mount('/content/drive')
import os 

!curl https://raw.githubusercontent.com/bxck75/colab_starter/main/setup.py -o /content/setup.py

!pip install tensorflow tensorflow-addons

######################################################

In [ ]:
!python /content/setup.py

In [ ]:
os.chdir('/content/GitGo/datasets')
!unzip /content/drive/MyDrive/garbage-20230421T030613Z-001.zip

In [ ]:
import glob,cv2
import numpy as np
from PIL import Image

!mkdir -p /content/GitGo/datasets/garbage_imgs_L
!mkdir -p /content/GitGo/datasets/garbage_imgs_P

i=0
for _f in sorted(glob.glob('/content/GitGo/datasets/garbage/*/*.jpg')):
  im = cv2.imread(_f)
  
  # You may need to convert the color.
  color_converted = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  pil_image=Image.fromarray(color_converted)
  
  h, w, c = im.shape
  split_fn = _f.split('/')
  print(split_fn[-1])
  if w < h:
    # portrait folder and rename to png
    pil_image.save('/content/GitGo/datasets/garbage_imgs_P/'+ split_fn[-1].replace('.jpg','.png'))
    i+=1
  else:
    # landscape folder and rename to png
    pil_image.save('/content/GitGo/datasets/garbage_imgs_L/'+ split_fn[-1].replace('.jpg','.png'))
    i+=1


In [ ]:
!rm -r /tmp/preprocess/*
!rm -r /content/GitGo/datasets/garbage_imgs_P/*
!rm -r /content/GitGo/datasets/garbage_imgs_L/*

In [ ]:
!rm -r /content/GitGo/datasets/garbage_imgs_P/*intro.png
!rm -r /content/GitGo/datasets/garbage_imgs_P/*_ch*.png

In [ ]:
import os
# Function to rename multiple files
def rename():
  i = 0
  path="/content/GitGo/datasets/garbage_imgs/"
  for _filename in sorted(os.listdir(path)):
    print(_filename)
    my_source = path + _filename
    my_dest = my_source.replace('.jpg','.png')
    # rename() function will
    # rename all the files
    os.rename(my_source, my_dest)
    i += 1


In [ ]:
!rm -r /tmp/preprocess/*

In [ ]:
import os
os.chdir('/content/GitGo/piss-ant-pix2pix')
input_images = '/content/GitGo/datasets/garbage_imgs_P'

In [ ]:

# resize
!python tools/process_custom.py --size 400 --pad --input_dir {input_images} --operation resize --output_dir /tmp/preprocess/_resized
# blank clones
#!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --operation blank --output_dir /tmp/preprocess/_blank
# edge clones
!mkdir -p /tmp/preprocess/_edges
!python /content/GitGo/piss-ant-pix2pix/tools/edge_custom.py --input_dir /tmp/preprocess/_resized --output_dir /tmp/preprocess/_edges
#!python tools/process_custom.py --input_dir /tmp/preprocess/_edge --operation blank --output_dir /tmp/preprocess/_blank_edge
# combine
#!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank --operation combine --output_dir  /tmp/preprocess/_resized/_combined_blank
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_edges --operation combine --output_dir  /tmp/preprocess/_resized/_combined_edge
#!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank_edge --operation combine --output_dir  /tmp/preprocess/_resized/_combined_blank_edge
# split
#!python tools/split.py --dir /tmp/preprocess/_resized/_combined_blank
!python tools/split.py --dir /tmp/preprocess/_resized/_combined_edge
#!python tools/split.py --dir /tmp/preprocess/_resized/_combined_blank_edge
# make outdirs
#!mkdir -p /content/GitGo/datasets/garbage_blank_dataset
!mkdir -p /content/GitGo/datasets/simpsons_edge_dataset
#!mkdir -p /content/GitGo/datasets/simpsons_blank_edge_dataset
# copy to final folders
#!cp -rf /tmp/preprocess/_resized/_combined_blank/ /content/GitGo/datasets/garbage_blank_dataset
!cp -rf /tmp/preprocess/_resized/_combined_edge/ /content/GitGo/datasets/garbage_edge_dataset
#!cp -rf /tmp/preprocess/_resized/_combined_blank_edge/ /content/GitGo/datasets/simpsons_blank_edge_dataset

##DeepDream

In [ ]:

import sys
if "/content/GitGo/TensorFlow-Tutorials" not in sys.path:
  sys.path.insert(1, "/content/GitGo/TensorFlow-Tutorials")
  print(sys.path)

%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
import math

import PIL.Image
from scipy.ndimage.filters import gaussian_filter
import inception5h
import importlib

importlib.reload(inception5h)


!mkdir -p /tmp/preprocess/_dream

inception5h.maybe_download()
model = inception5h.Inception5h()

#len(model.layer_tensors)
#print(model.layer_tensors)
#tensors = ['conv2d0:0', 'conv2d1:0', 'conv2d2:0', 'mixed3a:0', 'mixed3b:0', 'mixed4a:0', 'mixed4b:0', 'mixed4c:0', 'mixed4d:0', 'mixed4e:0', 'mixed5a:0', 'mixed5b:0']

##DreamMethods

In [ ]:
def load_image(filename):
    image = PIL.Image.open(filename)

    return np.float32(image)
    
def save_image(image, filename):
    # Ensure the pixel-values are between 0 and 255.
    image = np.clip(image, 0.0, 255.0)
    
    # Convert to bytes.
    image = image.astype(np.uint8)
    
    # Write the image-file in jpeg-format.
    with open(filename, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')

In [ ]:
def plot_image(image):
    # Assume the pixel-values are scaled between 0 and 255.
    
    if False:
        # Convert the pixel-values to the range between 0.0 and 1.0
        image = np.clip(image/255.0, 0.0, 1.0)
        
        # Plot using matplotlib.
        plt.imshow(image, interpolation='lanczos')
        plt.show()
    else:
        # Ensure the pixel-values are between 0 and 255.
        image = np.clip(image, 0.0, 255.0)
        
        # Convert pixels to bytes.
        image = image.astype(np.uint8)

        # Convert to a PIL-image and display it.
        display(PIL.Image.fromarray(image))
#Normalize an image so its values are between 0.0 and 1.0. This is useful for plotting the gradient.

def normalize_image(x):
    # Get the min and max values for all pixels in the input.
    x_min = x.min()
    x_max = x.max()

    # Normalize so all values are between 0.0 and 1.0
    x_norm = (x - x_min) / (x_max - x_min)
    
    return x_norm
#This function plots the gradient after normalizing it.

def plot_gradient(gradient):
    # Normalize the gradient so it is between 0.0 and 1.0
    gradient_normalized = normalize_image(gradient)
    
    # Plot the normalized gradient.
    plt.imshow(gradient_normalized, interpolation='bilinear')
    plt.show()
#This function resizes an image. It can take a size-argument where you give it the exact pixel-size you want the image to be e.g. (100, 200). Or it can take a factor-argument where you give it the rescaling-factor you want to use e.g. 0.5 for halving the size of the image in each dimension.

#This is implemented using PIL which is a bit lengthy because we are working on numpy arrays where the pixels are floating-point values. This is not supported by PIL so the image must be converted to 8-bit bytes while ensuring the pixel-values are within the proper limits. Then the image is resized and converted back to floating-point values.

def resize_image(image, size=None, factor=None):
    # If a rescaling-factor is provided then use it.
    if factor is not None:
        # Scale the numpy array's shape for height and width.
        size = np.array(image.shape[0:2]) * factor
        
        # The size is floating-point because it was scaled.
        # PIL requires the size to be integers.
        size = size.astype(int)
    else:
        # Ensure the size has length 2.
        size = size[0:2]
    
    # The height and width is reversed in numpy vs. PIL.
    size = tuple(reversed(size))

    # Ensure the pixel-values are between 0 and 255.
    img = np.clip(image, 0.0, 255.0)
    
    # Convert the pixels to 8-bit bytes.
    img = img.astype(np.uint8)
    
    # Create PIL-object from numpy array.
    img = PIL.Image.fromarray(img)
    
    # Resize the image.
    img_resized = img.resize(size, PIL.Image.LANCZOS)
    
    # Convert 8-bit pixel values back to floating-point.
    img_resized = np.float32(img_resized)

    return img_resized

In [ ]:
def get_tile_size(num_pixels, tile_size=400):
    """
    num_pixels is the number of pixels in a dimension of the image.
    tile_size is the desired tile-size.
    """

    # How many times can we repeat a tile of the desired size.
    num_tiles = int(round(num_pixels / tile_size))
    
    # Ensure that there is at least 1 tile.
    num_tiles = max(1, num_tiles)
    
    # The actual tile-size.
    actual_tile_size = math.ceil(num_pixels / num_tiles)
    
    return actual_tile_size
#This helper-function computes the gradient for an input image. The image is split into tiles and the gradient is calculated for each tile. The tiles are chosen randomly to avoid visible seams / lines in the final DeepDream image.

def tiled_gradient(gradient, image, tile_size=400):
    # Allocate an array for the gradient of the entire image.
    grad = np.zeros_like(image)

    # Number of pixels for the x- and y-axes.
    x_max, y_max, _ = image.shape

    # Tile-size for the x-axis.
    x_tile_size = get_tile_size(num_pixels=x_max, tile_size=tile_size)
    # 1/4 of the tile-size.
    x_tile_size4 = x_tile_size // 4

    # Tile-size for the y-axis.
    y_tile_size = get_tile_size(num_pixels=y_max, tile_size=tile_size)
    # 1/4 of the tile-size
    y_tile_size4 = y_tile_size // 4

    # Random start-position for the tiles on the x-axis.
    # The random value is between -3/4 and -1/4 of the tile-size.
    # This is so the border-tiles are at least 1/4 of the tile-size,
    # otherwise the tiles may be too small which creates noisy gradients.
    x_start = random.randint(-3*x_tile_size4, -x_tile_size4)

    while x_start < x_max:
        # End-position for the current tile.
        x_end = x_start + x_tile_size
        
        # Ensure the tile's start- and end-positions are valid.
        x_start_lim = max(x_start, 0)
        x_end_lim = min(x_end, x_max)

        # Random start-position for the tiles on the y-axis.
        # The random value is between -3/4 and -1/4 of the tile-size.
        y_start = random.randint(-3*y_tile_size4, -y_tile_size4)

        while y_start < y_max:
            # End-position for the current tile.
            y_end = y_start + y_tile_size

            # Ensure the tile's start- and end-positions are valid.
            y_start_lim = max(y_start, 0)
            y_end_lim = min(y_end, y_max)

            # Get the image-tile.
            img_tile = image[x_start_lim:x_end_lim,
                             y_start_lim:y_end_lim, :]

            # Create a feed-dict with the image-tile.
            feed_dict = model.create_feed_dict(image=img_tile)

            # Use TensorFlow to calculate the gradient-value.
            g = session.run(gradient, feed_dict=feed_dict)

            # Normalize the gradient for the tile. This is
            # necessary because the tiles may have very different
            # values. Normalizing gives a more coherent gradient.
            g /= (np.std(g) + 1e-8)

            # Store the tile's gradient at the appropriate location.
            grad[x_start_lim:x_end_lim,
                 y_start_lim:y_end_lim, :] = g
            
            # Advance the start-position for the y-axis.
            y_start = y_end

        # Advance the start-position for the x-axis.
        x_start = x_end

    return grad
#Optimize Image
#This function is the main optimization-loop for the DeepDream algorithm. It calculates the gradient of the given layer of the Inception model with regard to the input image. The gradient is then added to the input image so the mean value of the layer-tensor is increased. This process is repeated a number of times and amplifies whatever patterns the Inception model sees in the input image.

def optimize_image(layer_tensor, image,
                   num_iterations=10, step_size=3.0, tile_size=400,
                   show_gradient=False):
    """
    Use gradient ascent to optimize an image so it maximizes the
    mean value of the given layer_tensor.
    
    Parameters:
    layer_tensor: Reference to a tensor that will be maximized.
    image: Input image used as the starting point.
    num_iterations: Number of optimization iterations to perform.
    step_size: Scale for each step of the gradient ascent.
    tile_size: Size of the tiles when calculating the gradient.
    show_gradient: Plot the gradient in each iteration.
    """

    # Copy the image so we don't overwrite the original image.
    img = image.copy()
    
    print("Image before:")
    plot_image(img)

    print("Processing image: ", end="")

    # Use TensorFlow to get the mathematical function for the
    # gradient of the given layer-tensor with regard to the
    # input image. This may cause TensorFlow to add the same
    # math-expressions to the graph each time this function is called.
    # It may use a lot of RAM and could be moved outside the function.
    gradient = model.get_gradient(layer_tensor)
    
    for i in range(num_iterations):
        # Calculate the value of the gradient.
        # This tells us how to change the image so as to
        # maximize the mean of the given layer-tensor.
        grad = tiled_gradient(gradient=gradient, image=img,
                              tile_size=tile_size)
        
        # Blur the gradient with different amounts and add
        # them together. The blur amount is also increased
        # during the optimization. This was found to give
        # nice, smooth images. You can try and change the formulas.
        # The blur-amount is called sigma (0=no blur, 1=low blur, etc.)
        # We could call gaussian_filter(grad, sigma=(sigma, sigma, 0.0))
        # which would not blur the colour-channel. This tends to
        # give psychadelic / pastel colours in the resulting images.
        # When the colour-channel is also blurred the colours of the
        # input image are mostly retained in the output image.
        sigma = (i * 4.0) / num_iterations + 0.5
        grad_smooth1 = gaussian_filter(grad, sigma=sigma)
        grad_smooth2 = gaussian_filter(grad, sigma=sigma*2)
        grad_smooth3 = gaussian_filter(grad, sigma=sigma*0.5)
        grad = (grad_smooth1 + grad_smooth2 + grad_smooth3)

        # Scale the step-size according to the gradient-values.
        # This may not be necessary because the tiled-gradient
        # is already normalized.
        step_size_scaled = step_size / (np.std(grad) + 1e-8)

        # Update the image by following the gradient.
        img += grad * step_size_scaled

        if show_gradient:
            # Print statistics for the gradient.
            msg = "Gradient min: {0:>9.6f}, max: {1:>9.6f}, stepsize: {2:>9.2f}"
            print(msg.format(grad.min(), grad.max(), step_size_scaled))

            # Plot the gradient.
            plot_gradient(grad)
        else:
            # Otherwise show a little progress-indicator.
            print(". ", end="")

    print()
    print("Image after:")
    plot_image(img)
    
    return img
#Recursive Image Optimization
#The Inception model was trained on fairly small images. The exact size is unclear but maybe 200-300 pixels in each dimension. If we use larger images such as 1920x1080 pixels then the optimize_image() function above will add many small patterns to the image.

#This helper-function downscales the input image several times and runs each downscaled version through the optimize_image() function above. This results in larger patterns in the final image. It also speeds up the computation.

def recursive_optimize(layer_tensor, image,
                       num_repeats=4, rescale_factor=0.7, blend=0.2,
                       num_iterations=10, step_size=3.0,
                       tile_size=400):
    """
    Recursively blur and downscale the input image.
    Each downscaled image is run through the optimize_image()
    function to amplify the patterns that the Inception model sees.

    Parameters:
    image: Input image used as the starting point.
    rescale_factor: Downscaling factor for the image.
    num_repeats: Number of times to downscale the image.
    blend: Factor for blending the original and processed images.

    Parameters passed to optimize_image():
    layer_tensor: Reference to a tensor that will be maximized.
    num_iterations: Number of optimization iterations to perform.
    step_size: Scale for each step of the gradient ascent.
    tile_size: Size of the tiles when calculating the gradient.
    """

    # Do a recursive step?
    if num_repeats>0:
        # Blur the input image to prevent artifacts when downscaling.
        # The blur amount is controlled by sigma. Note that the
        # colour-channel is not blurred as it would make the image gray.
        sigma = 0.5
        img_blur = gaussian_filter(image, sigma=(sigma, sigma, 0.0))

        # Downscale the image.
        img_downscaled = resize_image(image=img_blur,
                                      factor=rescale_factor)
            
        # Recursive call to this function.
        # Subtract one from num_repeats and use the downscaled image.
        img_result = recursive_optimize(layer_tensor=layer_tensor,
                                        image=img_downscaled,
                                        num_repeats=num_repeats-1,
                                        rescale_factor=rescale_factor,
                                        blend=blend,
                                        num_iterations=num_iterations,
                                        step_size=step_size,
                                        tile_size=tile_size)
        
        # Upscale the resulting image back to its original size.
        img_upscaled = resize_image(image=img_result, size=image.shape)

        # Blend the original and processed images.
        image = blend * image + (1.0 - blend) * img_upscaled

    print("Recursive level:", num_repeats)

    # Process the image using the DeepDream algorithm.
    img_result = optimize_image(layer_tensor=layer_tensor,
                                image=image,
                                num_iterations=num_iterations,
                                step_size=step_size,
                                tile_size=tile_size)
    
    return img_result

##Dream images

In [ ]:
#session = tf.InteractiveSession(graph=model.graph)
tf.compat.v1.disable_eager_execution()
session = tf.compat.v1.Session(graph=model.graph)
def Go_Dream(folder, _tensor=3):
  
  for df in sorted(glob.glob(folder+'/*.png')):
    print(df)
    new_name = df.replace('_resized','_dream')
    print(new_name)
    #file_name = '/content/GitGo/datasets/garbage/llery_2k13mini/13mini_196b.jpg'
    image = load_image(filename=df)
    #plot_image(image)
    layer_tensor = model.layer_tensors[random.randrange(0, len(model.layer_tensors) - 1 )][:,:,:,0:3]
    layer_tensor
    img_result = recursive_optimize(layer_tensor=layer_tensor, image=image,
                    num_iterations=5, step_size=3.5, rescale_factor=0.902,
                    num_repeats=2, blend=0.005)
    
    layer_tensor = model.layer_tensors[random.randrange(0, len(model.layer_tensors) - 1 )]
    
    img_result_deep = optimize_image(layer_tensor, img_result,
                   num_iterations=5, step_size=2.0, tile_size=200,
                   show_gradient=False)
    nsplit = df.split('/')
   # new_name = df.replace('_resized','_dream')].replace('.png','.jpg')
    new_name ='/content/drive/MyDrive/DataSets/GPK/_dream/' + nsplit[-1].replace('.png','.jpg')
    print(new_name)
    save_image(img_result_deep, new_name)

Go_Dream('/tmp/preprocess/_resized',4)

## get faces only

In [ ]:
!cd /content/GitGo  && git clone https://github.com/kenxdrgn/Face-Detection-And-Auto-Crop

In [ ]:
!pip install -r /content/GitGo/Face-Detection-And-Auto-Crop/requirements.txt

In [ ]:
import os
os.chdir('/content/GitGo/Face-Detection-And-Auto-Crop/')
for g in sorted(glob.glob('/tmp/preprocess/_resized/*.png')):
  !python /content/GitGo/Face-Detection-And-Auto-Crop/auto_crop.py {g}

In [ ]:

from google.colab.patches import cv2_imshow
  
# Read the input image
img = cv2.imread('/content/GitGo/datasets/garbage_edge_dataset/val/13mini_197b.png')
  
# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

face_cascade = cv2.CascadeClassifier('/content/GitGo/opencv/data/haarcascades/haarcascade_frontalface_default.xml')
  
# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), 
                  (0, 0, 255), 2)
      
    face = img[y:y + h, x:x + w]
    cv2_imshow(face)
    cv2.imwrite('face.jpg', face)
  
cv2_imshow(img)
cv2.waitKey()

In [ ]:
!git clone https://github.com/opencv/opencv.git
!cd /content/GitGo/opencv

In [ ]:
os.chdir('/content/GitGo/opencv/data/lbpcascades')
!ls
#!curl https://raw.githubusercontent.com/aalind0/OpenCV-Anime-Face-Detector/master/Anime%20Face%20Detector/lbpcascade_animeface.xml -o lbpcascade_animeface.xml

In [ ]:
# Haar Cascade Image Feature Detection
import cv2
import numpy as np

# loading in the cascades.
face_cascade = cv2.CascadeClassifier('/content/GitGo/opencv/data/lbpcascades/lbpcascade_animeface.xml')

eye_cascade = cv2.CascadeClassifier('/content/GitGo/opencv/data/haarcascades/haarcascade_eye.xml')

# Reading the image.
img = cv2.imread('/content/drive/MyDrive/DataSets/GPK/_dream/13mini_197b.jpg')





# loading in the cascades.
face_cascade = cv2.CascadeClassifier('/content/GitGo/opencv/data/lbpcascades/lbpcascade_animeface.xml')

eye_cascade = cv2.CascadeClassifier('/content/GitGo/opencv/data/haarcascades/haarcascade_eye.xml')

# Reading the image.
img = cv2.imread('/content/GitGo/datasets/garbage_imgs_L/16promo_2.png')

# detection and drawing rectangles
while True:
    #converting the image to grayscale for easier processing.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0,255,0), 2)
      
    # Press 'ESC' to release the camera.        
    cv2_imshow(img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#!python /content/GitGo/piss-ant-pix2pix/tools/edge_custom.py --input_dir /tmp/preprocess/_resized --output_dir /tmp/preprocess/_edges

!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_edges --operation combine --output_dir  /tmp/preprocess/_combined_edge
!python tools/split.py --dir /tmp/preprocess/_combined_edge
!mkdir -p /content/GitGo/datasets/garbage_edge_dataset
!cp -rf /tmp/preprocess/_combined_edge/ /content/GitGo/datasets/garbage_edge_dataset

In [ ]:
%cd /content/GitGo/piss-ant-pix2pix
import cv2,glob
from google.colab.patches import cv2_imshow
!mkdir -p '/tmp/preprocess/_edges'


#for _f in sorted(glob.glob('/content/GitGo/datasets/garbage/*/*.jpg')):
def edge(_in,_out):
    image = cv2.imread(_in)
    #cv2_imshow(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # show the original and blurred images
    #cv2_imshow(image)
    #cv2_imshow( blurred)
    # compute a "wide", "mid-range", and "tight" threshold for the edges
    # using the Canny edge detector
    #wide = cv2.Canny(blurred, 10, 200)
    mid = cv2.Canny(blurred, 30, 150)
    #tight = cv2.Canny(blurred, 240, 250)
    # show the output Canny edge maps
    #cv2_imshow(~wide)
    #cv2_imshow(~mid)
    #cv2_imshow(~tight)
    print(_in)
    # You may need to convert the color.
    color_converted = cv2.cvtColor(~mid, cv2.COLOR_BGR2RGB)
    pil_image=Image.fromarray(color_converted)
    pil_image.save(_out)

    #cv2.imwrite(_out, ~mid)


for img_path in sorted(glob.glob('/tmp/preprocess/_resized/*/*.jpg')):
    print(img_path)
    new_img_path = img_path.replace('_resized','_edges')
    print(new_img_path)
    edge(img_path, new_img_path)


!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_edges --operation combine --output_dir  /tmp/preprocess/_combined_edge
!python tools/split.py --dir /tmp/preprocess/_combined_edge
!mkdir -p /content/GitGo/datasets/garbage_edge_dataset
!cp -rf /tmp/preprocess/_combined_edge/ /content/GitGo/datasets/garbage_edge_dataset

In [ ]:
print(os.path.exists('/tmp/preprocess/_edges'))
!ls /tmp/preprocess/_edges

In [ ]:
print(len( glob.glob('/tmp/preprocess/_edges/*.png')))
edges_imgs = glob.glob('/tmp/preprocess/_edges/*.png')

In [ ]:
!mkdir -p  /tmp/preprocess/_blank_edge
!python tools/process_custom.py --input_dir /tmp/preprocess/_edges --operation blank --output_dir /tmp/preprocess/_blank_edge
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank_edge --operation combine --output_dir  /tmp/preprocess/_combined_blank_edge
!python tools/split.py --dir /tmp/preprocess/_combined_blank_edge
!mkdir -p /content/GitGo/datasets/garbage_blank_edge_dataset
!cp -rf /tmp/preprocess/_combined_blank_edge /content/GitGo/datasets/garbage_blank_edge_dataset

In [ ]:
import glob
files = glob.glob('/content/GitGo/datasets/garbage_edge_dataset/_combined_edge/train/*.png')
print(len(files))

In [ ]:
!rm -r /content/drive/MyDrive/garbage_metrics/*

In [ ]:
!pip install -U tensorboard-plugin-profile
%reload_ext tensorboard
logs_base_dir = '/content/garbage_metrics'
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

In [ ]:
import os,warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings('ignore')
os.chdir('/content/GitGo/piss-ant-pix2pix')
custom_dataset = '/content/GitGo/datasets/garbage_blank_dataset/_combined_blank/train'
train_metrics = '/content/garbage_metrics'
!mkdir -p {train_metrics}

# CHECK IF ITS A RETRAINING
if os.path.exists('/content/simpsons_metrics/checkpoint'):
    retrain = '--checkpoint '+ train_metrics
else:
    retrain = ''
# train
!python pix2pix_custom.py \
    --summary_freq 50 \
    --ngf 64 \
    --ndf 64 \
    --save_freq 50 \
    --display_freq 10 \
    --progress_freq 50 \
    --batch_size 64 \
    --mode train \
    --output_dir {train_metrics} \
    --max_epochs 3 \
    --input_dir {custom_dataset} \
    {retrain} \
    --which_direction BtoA

In [ ]:
!rm -r /content/garbage_metrics/images

In [ ]:
!zip -r /content/drive/MyDrive/garbage_metrics_backup.zip /content/garbage_metrics